<a href="https://colab.research.google.com/gist/nanlliu/ccaa9beacfbb627b2da15790c0275fa6/clevr_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/MyDrive/clevr-dataset-gen

In [ ]:
%cd /content/gdrive/MyDrive/clevr-dataset-gen/image_generation

In [ ]:
!apt install blender
!apt install libboost-all-dev
!apt install libgl1-mesa-dev

In [ ]:
blender_version = 'blender2.79' #@param ["blender2.79", "blender2.80", "blender2.81", "blender2.82", "blender2.83", "blender2.90.1", "blender2.91.2"] {allow-input: false}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}

In [ ]:
if blender_version == "blender2.79":
    download_path="https://download.blender.org/release/Blender2.79/blender-2.79-linux-glibc219-x86_64.tar.bz2"
else:
  raise NotImplementedError

In [ ]:
!mkdir $blender_version
if blender_version == "blender2.79":
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1
else:
  raise NotImplementedError

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

In [ ]:
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

In [ ]:
%cd /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/ 
!echo $PWD >> blender2.79/2.79/python/lib/python3.5/site-packages/clevr.pth

In [ ]:
output_image_dir = YOUR_IMAGE_DIR
output_scene_dir = YOUR_SCENE_DIR
output_scene_file_path = YOUR_OVERALL_SCENE_FILE_PATH
output_blend_dir = YOUR_OUTPUT_BLENDER_DIR[OPTIONAL]

In [ ]:
!sudo ./$blender_version/blender \
--background --python render_images.py -- --num_images=9 --width=128 --height=128 \
--render_num_samples=64 --min_objects=1 --max_objects=5 --min_pixels_per_object=100 \
--output_image_dir=$output_image_dir \
--output_scene_dir=$output_scene_dir \
--output_scene_file=$output_scene_file_path \
--output_blend_dir=$output_blend_dir \
--min_dist=1 --margin=1 --use_gpu=1 &> /dev/null

In [ ]:
import os
import json
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

img_id = 0
img_path = os.path.join(output_image_dir, 'CLEVR_new_{:06}.png'.format(img_id))
im = Image.open(img_path).convert('RGB')
im = np.array(im)

scene_path = os.path.join(output_scene_dir, 'CLEVR_new_{:06}.json'.format(img_id))
with open(scene_path, 'r') as f:
  scene = json.load(f)

obj_names = []
for obj in scene['objects']:
  name = ' '.join([obj['size'], obj['color'], obj['material'], obj['shape']])
  obj_names.append(name)

for relation, indices in scene['relationships'].items():
  for i, js in enumerate(indices):
    for j in js:
      print(obj_names[j], relation, obj_names[i])

if len(obj_names) == 1:
  print(obj_names[0])

plt.figure()
plt.imshow(im)
plt.show()

In [ ]:
from PIL import Image
import torchvision
import torch
import numpy as np
from matplotlib import pyplot as plt

images = []

for i in range(9):
  img_path = os.path.join(output_image_dir, 'CLEVR_new_{:06}.png'.format(i))
  im = Image.open(img_path).convert('RGB')
  im = np.array(im).transpose(2, 0, 1)
  images.append(im)

images = torch.from_numpy(np.array(images))
grid = torchvision.utils.make_grid(images, nrow=3)

plt.figure(figsize = (20,20))
plt.imshow(grid.permute(1, 2, 0), interpolation='nearest')